# Simulator


&emsp;The goal of our work is researching some approaches to solve the problem of maximizing the average number of pages read in multi-page publications. We want to learn how we need to lead the agent through the pages to cause her consuming as much content as possible. Unfortunately we have no special lab or resource to perform field experiments. That's why we need to build a good simulator that will mimic the agents behavior.  
&emsp;The simulator receives from the outside directed graph that represents the structure of the content. Each vertex is a page and an edge from vertex A to vertex B expresses the link between page A and B, i.e. the next page that is presented to the agent after seeing page A may be page B. That is if there is no edge between vertex A and C , C can't be presented to the agent right after A.<br><br>
The core part of this project is imitating the traversal of the agent through the pages. This process is divided into rounds and on each round the following steps occurs:<br>
    &emsp;**1)**  Agent watches some page from the set of pages that we presented to her. In terms of graph abstraction - she is at some vertex of the graph.<br>
    &emsp;**2)** After watching the page, the agent chooses one of three actions: go to the next page, return to the previous page, or stop watching pages and leave.<br>
    &emsp;**3)** If the agent selected the "next" action, the simulator choose the page to present to the agent at the next round. <br>
    &emsp;&emsp; If the agent selected the "back" action, one of the pages she previously watched will be presented at the next round (the exact mechanics will be explained further).<br>
    &emsp;&emsp;If the agent selected the "leave" action, the simulation stops.<br>
    &emsp;**4)** The round finishes, the user is traversed to the node that was chosen on the previous step and  the next round starts.<br><br>

&emsp;Here focus on the first and the second step. We want to imitate the reaction of real Internet user to some piece of content. For example the agent liked the picture (or the sequence of pictures), found it to be interesting and want to see the next one, or the opposite case, when she is bored with the current picture (or sequence of presented pictures) and don't want to keep watching. It's important to mention that the action the user performs is not supposed to be  deterministic, it is more natural to think  of it in terms of probability. If the user liked the page she saw, she is more likely to press next, than after seeing the page that she loved less.  

    

### Simple version 

### Data structures and types

&emsp; In all versions of the simulator there will always be 3 entities: 1) The page with content - it is represented by node in graph 2) The graph that imposes constrains on the possible orders of  presenting the pages to the user 3)  The user that visits the pages .<br><br>

Here we present very simple implementation of the above concepts:
&emsp;Directed graph of the type *Graph1* describes possible pages links. Vertexes of the graph of type *Node1* represent one single page that the agent is supposed to see. The nodes has no special mechanism to describe the content of the associated page, and they differs only by id.<br>
<br>
&emsp; *User1* type represents  one agent, that traverse the pages of content. The core part of it is a *choiseFunction* attribute. It is a function that defines the behavior of the agent with respect to each page (whether she leaves, proceeds or goes back). It should be assigned when the user is created.

In [184]:
@enum UserAction  goBack goNext leave

struct Node1 
    id :: String
    NameInInput :: Any   
end


struct Graph1 
    graph :: Dict{Node1,Set{Node1}} 
end


struct User1 
   id :: String 
   ActionChoiseFunction  :: Any
end


### Auxiliary functions
Here we define simple auxiliary functions

In [185]:
# id_generator , get_node , printPath , getNeibhors

# Create a counter , used to produce unique ids
function id_generator()
    id = 0 
    return ()-> id = id + 1
end

nodeIdGenerator = id_generator()    # Nodes id generator
userIdGenerator = id_generator()    # Users id generator



# Return Node that corresponds to some name (that was given in input source , typicaly number)

function get_node(NameInInput, graph:: Graph1)
    nodes = graph.graph |> keys |> collect
    index = findfirst(node -> NameInInput == node.NameInInput,nodes) 
   return  nodes[index]
    end


# Given array of nodes , print their names devided by "=>"
# It is supposed to print the users path in the graph
function printPath(path :: Array{Node1,1})
    return   join(map(node-> node.NameInInput,path),"=>")
    
end

# Given node and graph , return node's neighbors
function getNeibhors(node :: Node1, graph :: Graph1) ::  Set{Node1}
        return graph.graph[node]
end;

### Producing the graph 

&emsp;The graph represents the structure of the content. Page B can be shown after page A only if there is edge (A,B) in the graph. It actually imposes constrains on the order in which pages can be shown to the user. <br>
Two extreme cases are possible : each node has outgoing and ingoing edges from/to all nodes -  all possible permutations are allowed. Or the line when each node has only one succedent - there is only one possible order.
<br>
The graph actually is not part of this simulator, it is used by an external algorithm that provides next node to the agent via special interface function (currently getNextNode). We need it here only to run the simulation.
 

The function below produces the simplest variant of the graph of type Graph1 from array of pairs of numbers. It doesn't have any distinctions between the nodes, all the pages are considered the same. It provides only the content structure - the edges between the nodes. 

In [186]:
function createGraph(simple_tuples :: Array{Tuple{Int64,Int64},1})
    
    node_dict = Dict()
    graph = Graph1(Dict())
    
    create_new_node = (NameInInput) -> 
                        (()-> (id = string(nodeIdGenerator()); Node1(id,NameInInput)))
    
    for (n1,n2) in simple_tuples
        
        node1 = get!(create_new_node(n1),node_dict,n1)
        
        node2 = get!(create_new_node(n2),node_dict,n2)
        
        neighbors = get!(Set,graph.graph,node1)
        get!(Set,graph.graph,node2)

        push!(neighbors,node2)      
        
    end

  return graph  
    
end
;

### Main interface functions

&emsp;The following functions are actually the most important parts of the simulator. The first one defines agent's behavior : which action will be chosen by the agent after watching some particular page (visiting some particular node ). <br>
&emsp;The second function defines which new page will be presented to the agent if she decided to proceed. It will connect the simulator to some external algorithm that will attempt to maximize the number of pages visited.



The functions below are simple implementations:<br>1) In order to choose the next action the inner function of the agent is called <br>2)The next page to show to the user is selected uniformly among neighbors of the current page in the graph. 

In [187]:



function chooseNextAction(user :: User1, node :: Node1) :: UserAction
    user.ActionChoiseFunction(node)
end



function getNextNode(graph :: Graph1, path :: Array{Node1,1})
    currentNode = path[end]
    neighbors = getNeibhors(currentNode,graph)
    if length(neighbors) == 0
        return currentNode
    else 
        return rand(neighbors)
    end
    
end;


## Main function 

&emsp;In *performWalk* function the simulation is executed for one agent. Given initial node and content graph, the following function simulates agent's traversal in the set of pages. It assumes that the agent can make 3 actions: 1) Press "next" -- proceed to the next page. 2) Press "back" - go back to the previous page 3) Leave the pages -- it is always the last action in the simulation for the current agent.<br>
In response to each action the current state of the simulator is updated , and the information about user's traversals is collected. <br>
There are two important records here : path and current node . <br> <br>
&emsp;1) **nodesVisited** -- list of unique nodes that were visited by the agent sorted by time they were revealed (by round index).<br>
Walking back and forth is allowed only along the *nodesVisited* list, it happens when the agent presses next and back. When the agent is at the last node ( nodesVisited\[end\] ) and press "next", she traverses to the new node that is returned by *getNextNode* function. <br> 
&emsp;2) **current node** -- the node the agent is currently at i.e the page the user watches now. <br><br>
&emsp;Example: at the beginning the path contains only the initial node N1 and the current node is also N1. Than the user presses "next" and the new node N2 is provided by *getNextNode*, the *nodesVisited* list is (N1,N2) and current node is N2. Than the user presses "back" , the *nodesVisited* remains the same and current node turns equal to N1 . Than the user presses "next" and the state turns to be *nodesVisited* =(N1,N2) current node = N2 .....

In [373]:

function performWalk(user :: User1, graph :: Graph1, initialNode :: Node1)
    
    currentNode = initialNode
    nodesVisited = Node1[currentNode]
    path = Node1[currentNode]
    currentIndex = 1
    stepCounter = 0

    while true
              
       nextAction = chooseNextAction(user, currentNode)
        
       # The agent is somewhere on previous pages that he visited and press next
        if nextAction == goNext && currentIndex < length(nodesVisited)
            currentIndex += 1
            currentNode = nodesVisited[currentIndex]
            stepCounter += 1
            push!(path,currentNode)
        # The agent is somewhere on previous pages that he visited and press back
        elseif nextAction == goBack && currentIndex > 1
            currentIndex -= 1
            currentNode = nodesVisited[currentIndex]
            stepCounter += 1
            push!(path,currentNode)
        # The agent is on the last page that he visited and press next
        elseif nextAction == goNext && currentIndex == length(nodesVisited)
            currentIndex += 1
            currentNode = getNextNode(graph,path)
            stepCounter += 1
            if nodesVisited[end] != currentNode
                push!(nodesVisited,currentNode)
            end
            push!(path,currentNode)
        # The agent press leave
        elseif nextAction == leave
            break
        # Unexpected combination of conditions
        else 
            stepCounter += 1

            push!(path,currentNode)
            # Throw exception
            # throw(ErrorException("Something went wrong path $path , current index $currentIndex , step counter $stepCounter, action $nextAction"))
       end
        
    end

    return Dict("path" => path, "nodesVisited" => nodesVisited)
end;


### The example of run 

1. ***Define agent's function for selecting the next action : simple uniform random selection***
+ Creating the user with this selection function
+ Creating a simple graph 
+ Getting the initial node object
+ ***Perform simulation***
+ Print the agents path 


In [393]:
chooseNextAction1 = x -> rand(instances(UserAction))

user = User1("1",chooseNextAction1)

graph = createGraph([(1,2),(2,3),(1,3),(3,4)])

initial_node = get_node(1,graph)

result = performWalk(user,graph,initial_node)

printPath(result["path"])

"1=>1=>3=>1=>1=>1=>3"